In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [2]:
import edward as ed
import numpy as np
import tensorflow as tf
import six
import os

In [3]:
from sklearn.linear_model import LogisticRegression

In [4]:
from edward.models import Normal,Empirical,Bernoulli,Categorical
from tensorflow.contrib import slim
from tensorflow.contrib.keras.api.keras.layers import Dense

In [5]:
import matplotlib.pyplot as plt
import train_utils as util

In [6]:
%matplotlib inline

In [7]:
mtype = 'bpm'
iftype = 'VI'

In [8]:
N = 10000  # number of data points
M = 3 # number of features
H = 3 # number of classes
B = 64 # batch size during training

In [9]:
s_mean = 0.
s_std = 1.

d_mean = 0.
d_std = 3.

noise_std = .01

In [10]:
Y,X = util.build_toy_dataset(mtype,N,M-1,H,s_std,s_mean,d_std)

In [11]:
X.shape

(10000, 2)

In [12]:
lgr = LogisticRegression()
lgr.fit(X,Y)
print('Logistic regression accuracy: ',sum(lgr.predict(X)==Y)/N)

Logistic regression accuracy:  0.9737


In [13]:
X = np.hstack((X,np.ones((N,1))))

In [14]:
X = X.astype(np.float32)
#Y = Y.astype(np.float32)

x_ph = tf.placeholder(tf.float32, [B,M])
y_ph = tf.placeholder(tf.int32,[B])
y_ph_ohe = tf.placeholder(tf.float32,[B,H]) 

In [15]:
# model
w = Normal(tf.zeros([H-1,M]),tf.ones([H-1,M]))
    
if iftype == 'HMC':
    qw = Empirical(params=tf.Variable(tf.random_normal([B,H-1,M])))
else:
    qw = Normal(tf.Variable(tf.random_normal([H-1,M])), tf.nn.softplus(tf.Variable(tf.random_normal([H-1,M]))))

#y = Categorical(tf.nn.softmax(Normal(tf.matmul(x_ph,tf.transpose(w)), noise_std)))
y = Categorical(tf.nn.softmax(Normal(tf.concat([tf.matmul(x_ph,tf.transpose(w)),tf.zeros([B,1])],axis=1), noise_std)))


In [16]:
tf.concat([tf.matmul(x_ph,tf.transpose(w)),tf.zeros([B,1])],axis=1)

<tf.Tensor 'concat_1:0' shape=(64, 3) dtype=float32>

In [17]:
tf.stack(qw)

<tf.Tensor 'Normal_1/sample/Reshape:0' shape=(2, 3) dtype=float32>

In [18]:
# inference
scaling = float(N) / B
nprint = 1000
niter = 20000

if iftype == 'EP':
    inference = ed.KLpq({w:qw},data={y:y_ph})
elif iftype == 'VI':
    inference = ed.KLqp({w:qw},data={y:y_ph})
elif iftype == 'HMC':
    inference = ed.HMC({w:qw},data={y:y_ph})
else:
    print('invalid inference type')
    
inference.initialize(n_iter=niter,n_print=nprint,scale={y:scaling})


In [19]:
sess = ed.get_session()
tf.global_variables_initializer().run()

In [20]:
ii = 0
sess = ed.get_session()
for t in range(niter):
    x_batch,y_batch,ii = util.get_next_batch(X,B,ii,Y)
  
    info_dict = inference.update(feed_dict={x_ph:x_batch,y_ph:y_batch})
    inference.print_progress(info_dict)
    
    if t % nprint == 0:
        print('\n w mean:')
        print(sess.run(qw))

    1/20000 [  0%]                                ETA: 5266s | Loss: 10028.760

train_utils.py:14: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if labels!=None:
train_utils.py:23: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if labels == None:



 w mean:
[[ 1.19152808 -1.8171078   0.92566633]
 [-0.89208293  1.22391331  0.94041562]]
 1000/20000 [  5%] █                              ETA: 14s | Loss: 6268.221 
 w mean:
[[ -3.16143012  -3.23294711  15.08413315]
 [ -1.00944471  -1.14759254   9.37395287]]
 2000/20000 [ 10%] ███                            ETA: 11s | Loss: 6375.683
 w mean:
[[ -3.3198359   -3.41533899  17.08827019]
 [ -1.23479724  -1.3176918   10.87611771]]
 3000/20000 [ 15%] ████                           ETA: 9s | Loss: 6550.994
 w mean:
[[ -3.35916471  -3.49550176  17.69033051]
 [ -1.27370107  -1.37511158  11.25664806]]
 4000/20000 [ 20%] ██████                         ETA: 8s | Loss: 6176.009
 w mean:
[[ -3.454566    -3.58475184  17.65071297]
 [ -1.23226678  -1.34309614  11.4538784 ]]
 5000/20000 [ 25%] ███████                        ETA: 8s | Loss: 6398.906
 w mean:
[[ -3.57357526  -3.79705477  17.7316227 ]
 [ -1.22293544  -1.30811608  11.29979897]]
 6000/20000 [ 30%] █████████                      ETA: 7s | Los

In [21]:
ii = 0
acu = 0
y_test = tf.nn.softmax(tf.concat([tf.matmul(x_ph,tf.transpose(qw.loc)),tf.zeros([B,1])],axis=1))
for i in range(int(np.ceil(N/B))):
    x_batch,y_batch,ii = util.get_next_batch(X,B,ii,Y)
    y_test_batch = sess.run(y_test,feed_dict={x_ph:x_batch,y_ph:y_batch})
    acu += sum(np.argmax(y_test_batch,axis=1)==y_batch)
print(acu*1./N)

0.9814


In [22]:
np.argmax(y_test_batch,axis=1)

array([1, 2, 2, 1, 0, 1, 0, 2, 2, 1, 2, 2, 1, 1, 2, 1, 1, 1, 0, 0, 2, 1, 0,
       2, 2, 1, 1, 1, 0, 1, 1, 0, 1, 2, 1, 2, 1, 1, 1, 1, 1, 0, 0, 2, 1, 2,
       2, 2, 1, 2, 2, 2, 0, 1, 2, 1, 2, 1, 1, 2, 2, 2, 1, 0])

In [23]:
y_batch

array([1, 2, 2, 1, 0, 1, 0, 2, 2, 1, 2, 2, 1, 1, 2, 1, 1, 1, 0, 0, 2, 1, 0,
       2, 2, 1, 1, 1, 0, 1, 1, 0, 1, 2, 1, 2, 1, 1, 2, 1, 1, 0, 0, 2, 1, 2,
       2, 2, 1, 2, 2, 2, 0, 1, 1, 1, 2, 1, 1, 2, 2, 2, 1, 0])